In [ ]:
#default_exp data.importer.buoy_data

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#export
from huey.imports import *

[2020-01-20 16:09:19,619] {settings.py:254} INFO - settings.configure_orm(): Using pool settings. pool_size=5, max_overflow=10, pool_recycle=1800, pid=24074


In [ ]:
realtime_url = "https://www.ndbc.noaa.gov/data/realtime2/46025.spec"
df = pd.read_csv(realtime_url, delim_whitespace=True)

In [ ]:
df.head()

,#YY,MM,DD,hh,mm,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
0,#yr,mo,dy,hr,mn,m,m,sec,m,sec,-,degT,-,sec,degT
1,2020,01,20,23,40,1.0,1.0,10.8,0.2,2.7,WSW,WSW,SWELL,8.6,252
2,2020,01,20,22,40,1.1,1.1,13.8,0.2,2.7,W,SW,SWELL,9.8,262
3,2020,01,20,21,40,1.1,1.1,12.1,0.2,2.6,WSW,WSW,SWELL,9.6,251
4,2020,01,20,20,40,1.1,1.1,13.8,0.2,2.9,W,SW,SWELL,9.4,267


# Import Buoy Realtime Wave Data

In [ ]:
station_id = "46025"

In [ ]:
realtime_url = f"https://www.ndbc.noaa.gov/data/realtime2/{station_id}.spec"

In [ ]:
realtime_url

'https://www.ndbc.noaa.gov/data/realtime2/46025.spec'

In [ ]:
df = pd.read_csv(realtime_url, delim_whitespace=True)

In [ ]:
df.head()

,#YY,MM,DD,hh,mm,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
0,#yr,mo,dy,hr,mn,m,m,sec,m,sec,-,degT,-,sec,degT
1,2020,01,20,23,40,1.0,1.0,10.8,0.2,2.7,WSW,WSW,SWELL,8.6,252
2,2020,01,20,22,40,1.1,1.1,13.8,0.2,2.7,W,SW,SWELL,9.8,262
3,2020,01,20,21,40,1.1,1.1,12.1,0.2,2.6,WSW,WSW,SWELL,9.6,251
4,2020,01,20,20,40,1.1,1.1,13.8,0.2,2.9,W,SW,SWELL,9.4,267


In [ ]:
df = df.replace('MM', np.NaN)

In [ ]:
df.tail()

,#YY,MM,DD,hh,mm,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
1074,2019,12,07,04,40,0.7,0.6,13.8,0.4,3.4,SW,S,NaN,5.4,223
1075,2019,12,07,03,40,0.8,0.7,13.8,0.4,3.1,S,S,NaN,5.3,183
1076,2019,12,07,02,40,0.7,0.6,14.8,0.3,3.0,S,SSE,NaN,4.9,188
1077,2019,12,07,01,40,0.6,0.6,14.8,0.3,2.4,SSW,SSE,NaN,5.7,196
1078,2019,12,07,00,40,0.6,0.6,13.8,0.1,3.7,S,S,NaN,7.3,172


In [ ]:
sql = f"select * from buoy_realtime_wave_detail order by ts desc limit 1"

In [ ]:
sql

'select * from buoy_realtime_wave_detail order by ts desc limit 1'

In [ ]:
from huey.data.sql import DB

In [ ]:
db = DB()
db_engine = db.engine

[2020-01-20 09:44:03,535] {base_hook.py:84} INFO - Using connection to: id: huey_dev. Host: localhost, Port: None, Schema: huey_dev, Login: postgres, Password: XXXXXXXX, extra: None


In [ ]:
result = pd.read_sql(sql, db_engine)

In [ ]:
result

,id,buoy_id,ts,significant_wave_height,swell_height,swell_period,swell_direction,wind_wave_height,wind_wave_period,steepness,average_wave_period,dominant_wave_direction,created_at,updated_at
0,5206,436,2020-01-17 15:40:00+00:00,1.4,1.1,7.1,WSW,0.8,4.5,STEEP,None,254,2020-01-17 17:00:10.118116+00:00,None


In [ ]:
df.head()

,#YY,MM,DD,hh,mm,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
0,#yr,mo,dy,hr,mn,m,m,sec,m,sec,-,degT,-,sec,degT
1,2020,01,20,16,40,1.0,0.9,13.8,0.3,2.9,W,SW,SWELL,7.2,262
2,2020,01,20,15,40,1.0,0.9,12.1,0.3,2.7,SW,SW,SWELL,7.0,233
3,2020,01,20,14,40,1.0,1.0,14.8,0.2,4.8,WSW,SW,SWELL,8.7,252
4,2020,01,20,13,40,1.0,1.0,14.8,0.2,2.2,W,SSW,SWELL,9.3,259


In [ ]:
len(df)

1096

In [ ]:
df2 = df.rename(columns={
    "WVHT": "significant_wave_height",
    "SwH": "swell_height",
    "SwP": "swell_period",
    "SwD": "swell_direction",
    "WWH": "wind_wave_height",
    "WWP": "wind_wave_period",
    "STEEPNESS": "steepness",
    "MWD": "dominant_wave_direction"
})

In [ ]:
df2.head()

,#YY,MM,DD,hh,mm,significant_wave_height,swell_height,swell_period,wind_wave_height,wind_wave_period,swell_direction,WWD,steepness,APD,dominant_wave_direction
0,#yr,mo,dy,hr,mn,m,m,sec,m,sec,-,degT,-,sec,degT
1,2020,01,20,16,40,1.0,0.9,13.8,0.3,2.9,W,SW,SWELL,7.2,262
2,2020,01,20,15,40,1.0,0.9,12.1,0.3,2.7,SW,SW,SWELL,7.0,233
3,2020,01,20,14,40,1.0,1.0,14.8,0.2,4.8,WSW,SW,SWELL,8.7,252
4,2020,01,20,13,40,1.0,1.0,14.8,0.2,2.2,W,SSW,SWELL,9.3,259


In [ ]:
df2 = df2[1:]

In [ ]:
df2.head()

,#YY,MM,DD,hh,mm,significant_wave_height,swell_height,swell_period,wind_wave_height,wind_wave_period,swell_direction,WWD,steepness,APD,dominant_wave_direction
1,2020,01,20,16,40,1.0,0.9,13.8,0.3,2.9,W,SW,SWELL,7.2,262
2,2020,01,20,15,40,1.0,0.9,12.1,0.3,2.7,SW,SW,SWELL,7.0,233
3,2020,01,20,14,40,1.0,1.0,14.8,0.2,4.8,WSW,SW,SWELL,8.7,252
4,2020,01,20,13,40,1.0,1.0,14.8,0.2,2.2,W,SSW,SWELL,9.3,259
5,2020,01,20,12,40,0.9,0.9,12.9,0.1,2.2,W,SSE,SWELL,10.0,261


In [ ]:
row = df2.iloc[0];
row

#YY                         2020
MM                            01
DD                            20
hh                            16
mm                            40
significant_wave_height      1.0
swell_height                 0.9
swell_period                13.8
wind_wave_height             0.3
wind_wave_period             2.9
swell_direction                W
WWD                           SW
steepness                  SWELL
APD                          7.2
dominant_wave_direction      262
Name: 1, dtype: object

In [ ]:
def ts_from_df_row(row):
        return datetime.datetime(
            int(row['#YY']), 
            int(row['MM']), 
            int(row['DD']), 
            int(row['hh']), 
            int(row['mm']), 
            tzinfo=datetime.timezone.utc)

In [ ]:
ts_from_df_row(row)

datetime.datetime(2020, 1, 20, 16, 40, tzinfo=datetime.timezone.utc)

In [ ]:
df2["ts"] = df2.apply(lambda row: ts_from_df_row(row), axis=1)

In [ ]:
df2.head()

,#YY,MM,DD,hh,mm,significant_wave_height,swell_height,swell_period,wind_wave_height,wind_wave_period,swell_direction,WWD,steepness,APD,dominant_wave_direction,ts
1,2020,01,20,16,40,1.0,0.9,13.8,0.3,2.9,W,SW,SWELL,7.2,262,2020-01-20 16:40:00+00:00
2,2020,01,20,15,40,1.0,0.9,12.1,0.3,2.7,SW,SW,SWELL,7.0,233,2020-01-20 15:40:00+00:00
3,2020,01,20,14,40,1.0,1.0,14.8,0.2,4.8,WSW,SW,SWELL,8.7,252,2020-01-20 14:40:00+00:00
4,2020,01,20,13,40,1.0,1.0,14.8,0.2,2.2,W,SSW,SWELL,9.3,259,2020-01-20 13:40:00+00:00
5,2020,01,20,12,40,0.9,0.9,12.9,0.1,2.2,W,SSE,SWELL,10.0,261,2020-01-20 12:40:00+00:00


In [ ]:
station_id = "46025"

In [ ]:
#export
from huey.data.sql import get_db_engine

In [ ]:
db_engine = get_db_engine()

[2020-01-20 16:10:48,423] {base_hook.py:84} INFO - Using connection to: id: huey_dev. Host: localhost, Port: None, Schema: huey_dev, Login: postgres, Password: XXXXXXXX, extra: None


In [ ]:
def get_latest_ob(station_id):
    sql = f"select * from buoy_realtime_wave_detail order by ts desc limit 1"
    result = pd.read_sql(sql, db_engine)
    return result


In [ ]:
res = get_latest_ob(station_id)

In [ ]:
res.ts

0   2020-01-17 15:40:00+00:00
Name: ts, dtype: datetime64[ns, UTC]

In [ ]:
#export
def ts_from_df_row(row):
        return datetime.datetime(
            int(row['#YY']), 
            int(row['MM']), 
            int(row['DD']), 
            int(row['hh']), 
            int(row['mm']), 
            tzinfo=datetime.timezone.utc)

In [ ]:
row = df.iloc[1];
ts_from_df_row(row)

datetime.datetime(2020, 1, 20, 23, 40, tzinfo=datetime.timezone.utc)

In [ ]:
#export
def import_buoy_realtime_wave_detail(station_id):
    db_engine = get_db_engine()
    sql = f"select * from buoy where station_id='{station_id}'"
    buoy = pd.read_sql(sql, db_engine).iloc[0]
    
    sql = f"select * from buoy_realtime_wave_detail order by ts desc limit 1"
    latest_ob = pd.read_sql(sql, db_engine, parse_dates=['ts']).iloc[0]
    
    station_id = "46025"
    realtime_url = f"https://www.ndbc.noaa.gov/data/realtime2/{station_id}.spec"
    df = pd.read_csv(realtime_url, delim_whitespace=True)
    
    df["buoy_id"] = buoy.id

    df = df.replace('MM', np.NaN)

    df2 = df.rename(columns={
        "WVHT": "significant_wave_height",
        "SwH": "swell_height",
        "SwP": "swell_period",
        "SwD": "swell_direction",
        "WWH": "wind_wave_height",
        "WWP": "wind_wave_period",
        "STEEPNESS": "steepness",
        "MWD": "dominant_wave_direction"
    })

    # skip first row
    df2 = df2[1:]

    def ts_from_df_row(row):
        return datetime.datetime(
            int(row['#YY']), 
            int(row['MM']), 
            int(row['DD']), 
            int(row['hh']), 
            int(row['mm']), 
            tzinfo=datetime.timezone.utc)

    df2["ts"] = df2.apply(lambda row: ts_from_df_row(row), axis=1)
    df2 = df2.drop(columns=['#YY', 'MM', 'DD', 'hh', 'mm'])
    df2 = df2.drop(columns=['WWD', 'APD'])
    
    sql = f"select * from buoy_realtime_wave_detail order by ts desc limit 1"
    latest_ob = pd.read_sql(sql, db_engine, parse_dates=['ts']).iloc[0]
    
    # only insert latest obs not in the db already
    df2_latest = df2[df2.ts > latest_ob.ts]
    
    df2_latest.to_sql('buoy_realtime_wave_detail', con=db_engine, if_exists='append', index=False)    

    return df2_latest

In [ ]:
latest_ob.ts

Timestamp('2020-01-17 15:40:00+0000', tz='UTC')

In [ ]:
df2 = import_buoy_realtime_wave_detail(station_id)

In [ ]:
df2.head()

,significant_wave_height,swell_height,swell_period,wind_wave_height,wind_wave_period,swell_direction,steepness,dominant_wave_direction,buoy_id,ts
1,1.1,1.1,12.1,0.2,4.0,WSW,SWELL,258,436,2020-01-21 00:40:00+00:00
2,1.0,1.0,10.8,0.2,2.7,WSW,SWELL,252,436,2020-01-20 23:40:00+00:00
3,1.1,1.1,13.8,0.2,2.7,W,SWELL,262,436,2020-01-20 22:40:00+00:00
4,1.1,1.1,12.1,0.2,2.6,WSW,SWELL,251,436,2020-01-20 21:40:00+00:00
5,1.1,1.1,13.8,0.2,2.9,W,SWELL,267,436,2020-01-20 20:40:00+00:00


In [ ]:
df2.tail()

,significant_wave_height,swell_height,swell_period,wind_wave_height,wind_wave_period,swell_direction,steepness,dominant_wave_direction,buoy_id,ts
77,1.1,1.0,10.0,0.3,3.7,WSW,SWELL,254,436,2020-01-17 20:40:00+00:00
78,1.1,1.1,5.9,0.4,3.8,WSW,STEEP,245,436,2020-01-17 19:40:00+00:00
79,1.2,1.1,7.1,0.4,3.3,WSW,AVERAGE,243,436,2020-01-17 18:40:00+00:00
80,1.1,1.0,5.3,0.4,3.8,WSW,VERY_STEEP,246,436,2020-01-17 17:40:00+00:00
81,1.2,1.1,7.7,0.4,3.4,WSW,AVERAGE,248,436,2020-01-17 16:40:00+00:00


In [ ]:
df3 = import_buoy_realtime_wave_detail(station_id)

In [ ]:
len(df3)

0

# Export

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.sql.ipynb.
Converted 02_data.importer.buoy_data.ipynb.
